In [1]:
import numpy as np
import pandas as pd
import scipy.stats

pd.options.mode.chained_assignment = None
from matplotlib import pyplot as plt, rcParams
# import cv2
import seaborn as sns

sns.set(style="white", context="paper")
from cycler import cycler
import os, sys
import glob
from datetime import datetime, timedelta
from itertools import combinations, product
import base64
from PIL import Image
from io import BytesIO as _BytesIO
import requests
import json
import pickle
from datetime import datetime
from IPython.display import display, Markdown, Latex
from sklearn.metrics import *
import collections
from copy import deepcopy
import traceback
from sympy import Point, Polygon
from decorators import *
from smartprint import smartprint as sprint
from scipy.spatial.distance import cdist
from sklearn.cluster import DBSCAN
import mmcv
from mmtrack.apis import inference_mot, init_model as init_tracking_model
# import plotly
# from pandas_profiling import ProfileReport

pd.options.display.max_columns = None
def printm(s): return display(Markdown(s))
    
SERVER_CACHE_DIR = '/mnt/ci-nas-cache/edulyzeV2/cache_compute_4/fixed_face'
os.makedirs(SERVER_CACHE_DIR,exist_ok=True)

track_analysis_meta_cache = f'{SERVER_CACHE_DIR}/analysis_tracking/meta_info'
base_dir = '/mnt/ci-nas-cache/edulyzeV2/pose_face_gaze_emb_fixed_face/'

track_analysis_session_data = f'{SERVER_CACHE_DIR}/analysis_tracking/session_tracking_info'
os.makedirs(track_analysis_session_data,exist_ok=True)

postprocessed_id_map_data_dir = f'{SERVER_CACHE_DIR}/analysis_tracking/processed_id_maps'
os.makedirs(postprocessed_id_map_data_dir, exist_ok=True)

emb_analysis_session_data = f'{SERVER_CACHE_DIR}/analysis_emb/session_emb_info_new'
os.makedirs(emb_analysis_session_data,exist_ok=True)

embmatched_id_raw_data_dir = f'{SERVER_CACHE_DIR}/analysis_emb/embmatched_id_raw'
os.makedirs(embmatched_id_raw_data_dir,exist_ok=True)

embmatched_id_map_data_dir = f'{SERVER_CACHE_DIR}/analysis_tracking/embmatched_id_maps_new'
os.makedirs(embmatched_id_map_data_dir, exist_ok=True)

cross_session_input_data_dir = f'{SERVER_CACHE_DIR}/analysis_emb/cross_session_input'
os.makedirs(cross_session_input_data_dir, exist_ok=True)

id_viz_cache_root = f'{SERVER_CACHE_DIR}/analysis_emb/session_matching_info'
os.makedirs(id_viz_cache_root, exist_ok=True)

v5_id_start_stop_data_dir = f'{SERVER_CACHE_DIR}/analysis_emb/v5_id_start_stop_info'
os.makedirs(v5_id_start_stop_data_dir, exist_ok=True)

/home/prasoon/anaconda3/envs/edusense/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/prasoon/anaconda3/envs/edusense/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/prasoon/anaconda3/envs/edusense/lib/python3.9/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


## Get frame file data for all sessions

In [2]:

frame_file_data = {}
for course_idx, course_dir in enumerate(glob.glob(f"{base_dir}/*")):
    course_name = course_dir.split("/")[-1]
    course_cache_file = f"{track_analysis_meta_cache}/{course_name}"
    if os.path.exists(course_cache_file):
        frame_file_data[course_name] = pickle.load(open(course_cache_file,"rb"))
        continue
    frame_file_data[course_name]={}
        
    for session_idx, session_dir in enumerate(glob.glob(f"{course_dir}/*")):
        session_name = session_dir.split("/")[-1]
        frame_file_data[course_name][session_name] = {}
        frame_files = glob.glob(f"{session_dir}/*")
        frame_file_names = [xr.split("/")[-1] for xr in frame_files]
        if 'end.pb' in frame_file_names:
            frame_file_data[course_name][session_name]['is_completed']=True
        else:
            frame_file_data[course_name][session_name]['is_completed']=False            
        frame_ids = [int(xr.split(".")[0]) for xr in frame_file_names if not (xr=='end.pb')]
        frame_file_data[course_name][session_name]['frame_ids'] = sorted(frame_ids)
        frame_file_data[course_name][session_name]['dir_location'] = session_dir
        print(f"Got metadata for course: {course_idx}-{course_name}, session:{session_idx}-{session_name}")
    pickle.dump(frame_file_data[course_name],open(course_cache_file,"wb")) 
        
frame_file_data.keys()

dict_keys(['05748A', '05410B', '17214C', '17214B', '05681A', '24352A', '24352C', '86375A'])

In [3]:
session_filter_list = [
 #    'classinsight-cmu_05681A_ghc_4301_201905011630',
 # 'classinsight-cmu_05681A_ghc_4301_201904171630',
 # 'classinsight-cmu_05681A_ghc_4301_201902201630',
 # 'classinsight-cmu_05681A_ghc_4301_201904101630',
 # 'classinsight-cmu_05681A_ghc_4301_201901231630',
                       
 # 'classinsight-cmu_05418A_ghc_4102_201902251200',
 # 'classinsight-cmu_05418A_ghc_4102_201904081200',
 # 'classinsight-cmu_05418A_ghc_4102_201905011200',
 # 'classinsight-cmu_05418A_ghc_4102_201904291200',
 # 'classinsight-cmu_05418A_ghc_4102_201904011200',
                       
 'classinsight-cmu_05748A_ghc_4101_201902141630',
 'classinsight-cmu_05748A_ghc_4101_201904021630',
 'classinsight-cmu_05748A_ghc_4101_201902051630',
 'classinsight-cmu_05748A_ghc_4101_201902281630',
 'classinsight-cmu_05748A_ghc_4101_201903071630',
                       
 # 'classinsight-cmu_21127J_ghc_4102_201904230930',
 # 'classinsight-cmu_21127J_ghc_4102_201903260930',
 # 'classinsight-cmu_21127J_ghc_4102_201904160930',
 # 'classinsight-cmu_21127J_ghc_4102_201904300930',
 # 'classinsight-cmu_21127J_ghc_4102_201903190930',
                       
 # 'classinsight-cmu_05410A_ghc_4301_201904151500',
 # 'classinsight-cmu_05410A_ghc_4301_201902251500',
 # 'classinsight-cmu_05410A_ghc_4301_201904081500',
 # 'classinsight-cmu_05410A_ghc_4301_201904221500',
 # 'classinsight-cmu_05410A_ghc_4301_201902181500',
                       
 'classinsight-cmu_17214B_ph_a21_201902271030',
 'classinsight-cmu_17214B_ph_a21_201903061030',
 'classinsight-cmu_17214B_ph_a21_201904031030',
 'classinsight-cmu_17214B_ph_a21_201904101030',
 'classinsight-cmu_17214B_ph_a21_201904241030',
                       
 'classinsight-cmu_17214C_ph_225b_201903201130',
 'classinsight-cmu_17214C_ph_225b_201904101130',
 'classinsight-cmu_17214C_ph_225b_201904171130',
 'classinsight-cmu_17214C_ph_225b_201904241130',
 'classinsight-cmu_17214C_ph_225b_201905011130',
                       
 # 'classinsight-cmu_05410B_ghc_4211_201902111500',
 # 'classinsight-cmu_05410B_ghc_4211_201903181500',
 # 'classinsight-cmu_05410B_ghc_4211_201904081500',
 # 'classinsight-cmu_05410B_ghc_4211_201904151500',
 # 'classinsight-cmu_05410B_ghc_4211_201904221500',
 # 'classinsight-cmu_05410B_ghc_4211_201901281500'
]

# Get frames data across all sessions

In [4]:
frame_file_data = {}
for course_idx, course_dir in enumerate(glob.glob(f"{base_dir}/*")):
    course_name = course_dir.split("/")[-1]
    course_cache_file = f"{track_analysis_meta_cache}/{course_name}"
    if os.path.exists(course_cache_file):
        frame_file_data[course_name] = pickle.load(open(course_cache_file,"rb"))
        continue
    frame_file_data[course_name]={}
        
    for session_idx, session_dir in enumerate(glob.glob(f"{course_dir}/*")):
        session_name = session_dir.split("/")[-1]
        frame_file_data[course_name][session_name] = {}
        frame_files = glob.glob(f"{session_dir}/*")
        frame_file_names = [xr.split("/")[-1] for xr in frame_files]
        if 'end.pb' in frame_file_names:
            frame_file_data[course_name][session_name]['is_completed']=True
        else:
            frame_file_data[course_name][session_name]['is_completed']=False            
        frame_ids = [int(xr.split(".")[0]) for xr in frame_file_names if not (xr=='end.pb')]
        frame_file_data[course_name][session_name]['frame_ids'] = sorted(frame_ids)
        frame_file_data[course_name][session_name]['dir_location'] = session_dir
        print(f"Got metadata for course: {course_idx}-{course_name}, session:{session_idx}-{session_name}")
    pickle.dump(frame_file_data[course_name],open(course_cache_file,"wb")) 
        
frame_file_data.keys()

dict_keys(['05748A', '05410B', '17214C', '17214B', '05681A', '24352A', '24352C', '86375A'])

# Get id_start_stop info for all sessions

In [5]:
# writing a generic loop to get embedding info from all courses in frame file data

for course_idx, course in enumerate(frame_file_data):
    for session_idx, session_id in enumerate(frame_file_data[course]):
        if session_id.split("-front")[0] not in session_filter_list:
            print(f"Session {session_id} not in session filter list, skipping...")
            continue
        v5_id_start_stop_cache_file = f"{v5_id_start_stop_data_dir}/{session_id}.csv"
        try:
            if not os.path.exists(v5_id_start_stop_cache_file):
                session_dir = frame_file_data[course][session_id]['dir_location']
                frame_ids = frame_file_data[course][session_id]['frame_ids']
                
                session_tracking_cache_file = f"{track_analysis_session_data}/{session_id}.pb"
                session_preprocessed_id_map_file = f"{postprocessed_id_map_data_dir}/{session_id}.pb"
                session_eligible_pairs_map_file = f"{embmatched_id_map_data_dir}/{session_id}.csv"

                printm("### get final id mapping for old to new ids with emb processing.")
                old_to_new_id_map = pickle.load(open(session_preprocessed_id_map_file,"rb"))
                df_session_eligible_pairs = pd.read_csv(session_eligible_pairs_map_file)
                
                printm("### Correct new ids with dict from eligible pairs")
                eligible_id_map_dict = {}
                for id_pair in df_session_eligible_pairs.id_pair.values:
                    (id1, id2) = eval(id_pair)
                    print(id1, id2)
                    if id2 in eligible_id_map_dict:
                        eligible_id_map_dict[id1] = eligible_id_map_dict[id2]
                    else:
                        eligible_id_map_dict[id2] = id1
                
                sprint(eligible_id_map_dict)
                
                printm("### correct created eligible map for once more")
                for key in sorted(list(eligible_id_map_dict.keys())):
                    key_value = eligible_id_map_dict[key]
                    if key_value in eligible_id_map_dict.keys():
                        eligible_id_map_dict[key] = eligible_id_map_dict[key_value]
                sprint(eligible_id_map_dict) 
                # sprint({kr:old_to_new_id_map[kr] for kr in old_to_new_id_map if (not old_to_new_id_map[kr]==10000)}) 
                
                sprint("replacing ids in old to new id maps")
                for old_id in old_to_new_id_map:
                    if old_to_new_id_map[old_id] in eligible_id_map_dict:
                        print(f"replacing {old_id}:{old_to_new_id_map[old_id]} -->{eligible_id_map_dict[old_to_new_id_map[old_id]]}")
                        old_to_new_id_map[old_id] = eligible_id_map_dict[old_to_new_id_map[old_id]]
                # sprint({kr:old_to_new_id_map[kr] for kr in old_to_new_id_map if (not old_to_new_id_map[kr]==10000)}) 

                printm("### map new ids to tracking dataframe")
                df_tracking_new = pickle.load(open(session_tracking_cache_file,"rb")).transpose()
                total_idxs = df_tracking_new.index.max()
                for old_id in old_to_new_id_map:
                    new_id = old_to_new_id_map[old_id]
                    if not new_id==10000:
                        new_id_col = f'N{new_id}'
                        if new_id_col not in df_tracking_new:
                            df_tracking_new[new_id_col] = None
                        df_tracking_new[new_id_col] =  df_tracking_new[new_id_col].where(~df_tracking_new[new_id_col].isnull(), df_tracking_new[old_id])
                    df_tracking_new = df_tracking_new.drop(old_id, axis=1)

                printm("### get id start stop dataframe")
                col_start_stop_idxs = []
                for col in df_tracking_new.columns:
                    one_idxs = df_tracking_new.index[np.where(df_tracking_new[col]==1)[0]].values
                    col_start_stop_idxs.append([col, one_idxs.min(), one_idxs.max()])
                df_id_start_stop = pd.DataFrame(col_start_stop_idxs, columns=['id','min_idx','max_idx'])
                df_id_start_stop['total_idxs'] = df_id_start_stop['max_idx']-df_id_start_stop['min_idx']
                df_id_start_stop['id'] = df_id_start_stop['id'].apply(lambda x: int(x[1:]))
                df_id_start_stop.to_csv(v5_id_start_stop_cache_file, index=False)
                printm(f"## Got id start stop info for session: {course_idx}-{course}, session:{session_idx}-{session_id}")
            else:
                ...
                printm(f"### FILE EXISTS: id start stop info for session: {course_idx}-{course}, session:{session_idx}-{session_id}")
        except:
            printm(f"## ERROR: Unable to get id start stop info for: {course_idx}-{course}, session:{session_idx}-{session_id}")
            unfinished_sessions.append((course, session_id))
            print(traceback.format_exc())
    

### FILE EXISTS: id start stop info for session: 0-05748A, session:0-classinsight-cmu_05748A_ghc_4101_201902051630-front

### FILE EXISTS: id start stop info for session: 0-05748A, session:1-classinsight-cmu_05748A_ghc_4101_201902141630-front

Session classinsight-cmu_05748A_ghc_4101_201901291630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201901311630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201902071630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201902121630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201902191630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201902211630-front not in session filter list, skipping...


### FILE EXISTS: id start stop info for session: 0-05748A, session:8-classinsight-cmu_05748A_ghc_4101_201902281630-front

Session classinsight-cmu_05748A_ghc_4101_201903051630-front not in session filter list, skipping...


...

### FILE EXISTS: id start stop info for session: 3-17214B, session:5-classinsight-cmu_17214B_ph_a21_201904031030-front

### FILE EXISTS: id start stop info for session: 3-17214B, session:6-classinsight-cmu_17214B_ph_a21_201904101030-front

Session classinsight-cmu_17214B_ph_a21_201904171030-front not in session filter list, skipping...
Session classinsight-cmu_17214B_ph_a21_201902131045-front not in session filter list, skipping...


### FILE EXISTS: id start stop info for session: 3-17214B, session:9-classinsight-cmu_17214B_ph_a21_201904241030-front

Session classinsight-cmu_17214B_ph_a21_201905011030-front not in session filter list, skipping...
Session classinsight-cmu_05681A_ghc_4301_201901301630-front not in session filter list, skipping...
Session classinsight-cmu_05681A_ghc_4301_201902061630-front not in session filter list, skipping...
Session classinsight-cmu_05681A_ghc_4301_201902131633-front not in session filter list, skipping...
Session classinsight-cmu_05681A_ghc_4301_201902201630-front not in session filter list, skipping...
Session classinsight-cmu_05681A_ghc_4301_201902271630-front not in session filter list, skipping...
Session classinsight-cmu_05681A_ghc_4301_201902271718-front not in session filter list, skipping...
Session classinsight-cmu_05681A_ghc_4301_201903131630-front not in session filter list, skipping...
Session classinsight-cmu_05681A_ghc_4301_201903201630-front not in session filter list, skipping...
Session classinsight-cmu_05681A_ghc_4301_201903271630-front not in session filter list, skipping...
Se

# Get Groundtruth for course

In [360]:
course = '17214C'
df_gt = pd.read_csv(f"groundtruth/{course}.tsv",sep='\t',index_col=0)
df_gt

,Session-0410,Session-0417,Session-0424,Session-0501,Session-0320
CourseID,,,,,
1,0,"3,16",1,5,2
2,"1,14",-1,-1,-1,3
3,2,13,13,2,8
4,3,-1,-1,-1,0
5,4,2,11,"16,6",4
6,6,1,6,13,-1
7,"7,16",5,9,7,9
8,8,8,2,10,-1
9,"9,13",14,5,11,13


In [361]:
# get GT pairs

In [362]:
# convert ground truth into gt_pairs


# New algorithm for session pair id matching based on consistencies of ids present

## Describing the algorithm here.

We are trying to find consistent ids across all sessions, and only do matching for those ids.

- Step 1: filter out ids which are consistent more than 75% of the session from both sessions.
- Step 2: Use 3 tiered rules to match ids across those sessions based on gaze and clu matches.

In [363]:
# get input for all sessions
course_input_dict = {}
idstartstop_input_dict = {}
sessions = [xr for xr in session_filter_list if (course in xr)]
for session in sessions:
    session_input_file = f'{cross_session_input_data_dir}/{session}-front.pb'
    session_id_start_stop_file = f'{v5_id_start_stop_data_dir}/{session}-front.csv'
    course_input_dict[session] = pickle.load(open(session_input_file,"rb"))
    idstartstop_input_dict[session] = pd.read_csv(session_id_start_stop_file)
course_input_dict.keys()

dict_keys(['classinsight-cmu_17214C_ph_225b_201903201130', 'classinsight-cmu_17214C_ph_225b_201904101130', 'classinsight-cmu_17214C_ph_225b_201904171130', 'classinsight-cmu_17214C_ph_225b_201904241130', 'classinsight-cmu_17214C_ph_225b_201905011130'])

In [364]:
df_filtered_gt = deepcopy(df_gt)
for col in df_filtered_gt:
    df_filtered_gt[col] = df_filtered_gt[col].apply(lambda x: list(map(int,str(x).split(","))) if not (x=='-1') else [])
df_filtered_gt

,Session-0410,Session-0417,Session-0424,Session-0501,Session-0320
CourseID,,,,,
1,[0],"[3, 16]",[1],[5],[2]
2,"[1, 14]",[],[-1],[],[3]
3,[2],[13],[13],[2],[8]
4,[3],[],[-1],[],[0]
5,[4],[2],[11],"[16, 6]",[4]
6,[6],[1],[6],[13],[]
7,"[7, 16]",[5],[9],[7],[9]
8,[8],[8],[2],[10],[]
9,"[9, 13]",[14],[5],[11],[13]


In [365]:
course_input_dict_raw= deepcopy(course_input_dict)

for session in sessions:
    df_id_start_stop_session = idstartstop_input_dict[session]
    df_id_start_stop_session['presence_fraction'] = df_id_start_stop_session.total_idxs / df_id_start_stop_session.total_idxs.max()
    filtered_ids = df_id_start_stop_session[df_id_start_stop_session.presence_fraction>0.5]['id'].values
    course_input_dict[session] = {xr: course_input_dict[session][xr] for xr in course_input_dict[session] if xr in filtered_ids}
    session_col = f'Session-{session[-8:-4]}'
    df_filtered_gt[session_col] = df_filtered_gt[session_col].apply(lambda x: [xr for xr in x if (xr in filtered_ids)])
    sprint(session, course_input_dict[session].keys())
    # sprint(df_id_start_stop_session)
    # _ = plt.figure(figsize=(20,15))
    # for row_idx, row in df_id_start_stop_session.iterrows():
    #     plt.axhline(y=row_idx, xmin=row['min_idx']/df_id_start_stop_session.total_idxs.max(),xmax=row['max_idx']/df_id_start_stop_session.total_idxs.max())
    # plt.yticks(range(df_id_start_stop_session.shape[0]), range(df_id_start_stop_session.shape[0]))
    # plt.grid() 
    sprint(filtered_ids)
df_filtered_gt    
    

session, course_input_dict[session].keys() : classinsight-cmu_17214C_ph_225b_201903201130 dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
filtered_ids : [ 0  1  2  3  4  5  6  7  8  9 10]
session, course_input_dict[session].keys() : classinsight-cmu_17214C_ph_225b_201904101130 dict_keys([0, 1, 2, 3, 4, 6, 7, 11, 13])
filtered_ids : [ 0  1  2  3  4  6  7 11 13]
session, course_input_dict[session].keys() : classinsight-cmu_17214C_ph_225b_201904171130 dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 18])
filtered_ids : [ 0  1  2  3  4  5  6  7  8 10 11 13 14 18]
session, course_input_dict[session].keys() : classinsight-cmu_17214C_ph_225b_201904241130 dict_keys([0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13])
filtered_ids : [ 0  1  2  3  4  5  6  8  9 10 11 12 13]
session, course_input_dict[session].keys() : classinsight-cmu_17214C_ph_225b_201905011130 dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13])
filtered_ids : [ 0  1  2  3  4  5  6  7  9 10 11 12 13]


,Session-0410,Session-0417,Session-0424,Session-0501,Session-0320
CourseID,,,,,
1,[0],[3],[1],[5],[2]
2,[1],[],[],[],[3]
3,[2],[13],[13],[2],[8]
4,[3],[],[],[],[0]
5,[4],[2],[11],[6],[4]
6,[6],[1],[6],[13],[]
7,[7],[5],[9],[7],[9]
8,[],[8],[2],[10],[]
9,[13],[14],[5],[11],[]


In [366]:
# get possible gt matches
true_matches = []
session_cols = list(df_filtered_gt.columns)
for idx_colA in range(len(session_cols)):
    for idx_colB in range(idx_colA+1, len(session_cols)):
        match_val_pairs = zip(df_filtered_gt[session_cols[idx_colA]].values.tolist(), df_filtered_gt[session_cols[idx_colB]].values.tolist())
        for ids_colA, ids_colB in match_val_pairs:
            for id_colA in ids_colA:
                for id_colB in ids_colB:
                    session_keyA = session_cols[idx_colA].split("-")[-1]
                    session_keyB = session_cols[idx_colB].split("-")[-1]
                    true_matches.append([session_keyA, session_keyB, f'{session_keyA}_{id_colA}', f'{session_keyB}_{id_colB}'])
df_true_matches = pd.DataFrame(true_matches, columns=['sessionA','sessionB','idA','idB'])
df_true_matches['gt'] = 1
df_true_matches             

,sessionA,sessionB,idA,idB,gt
0,0410,0417,0410_0,0417_3,1
1,0410,0417,0410_2,0417_13,1
2,0410,0417,0410_4,0417_2,1
3,0410,0417,0410_6,0417_1,1
4,0410,0417,0410_7,0417_5,1
...,...,...,...,...,...
60,0501,0320,0501_2,0320_8,1
61,0501,0320,0501_6,0320_4,1
62,0501,0320,0501_7,0320_9,1
63,0501,0320,0501_4,0320_1,1


In [367]:
course_input_dict[session].keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13])

In [368]:
type='constrained'

In [369]:
# matching ids0.40	0.3	50
# original 0.3, 0.2, 40
MATCH_THRESHOLD=0.35
MATCH_PAIR_MAX_THRESHOLD = 0.2
MATCH_PAIR_AREA_THRESHOLD = 20

# unconstrained
if type=='unconstrained':
    MATCH_THRESHOLD=0.35
    MATCH_PAIR_MAX_THRESHOLD = 0.2
    MATCH_PAIR_AREA_THRESHOLD = 20
    print(type, MATCH_THRESHOLD, MATCH_PAIR_MAX_THRESHOLD, MATCH_PAIR_AREA_THRESHOLD)

final_matches = []
for (sessionA, sessionB) in product(sorted(sessions), sorted(sessions)):
    sessionA_key, sessionB_key = sessionA.split("_")[-1][4:8], sessionB.split("_")[-1][4:8]
    if not (sessionA==sessionB):
        # if sessionA not in session_matches:
        # match session A and session B based on gaze clustering
        match_scores_gaze = {}
        match_scores_clu  = {}
        for idA,idB in product(course_input_dict[sessionA].keys(), course_input_dict[sessionB].keys()):

            if not (type=='unconstrained'):
                # check if there exists a gt match for idA in SessionB
                idA_key = f'{sessionA_key}_{idA}'
                idB_key = f'{sessionB_key}_{idB}'
                idA_matches = df_true_matches[(df_true_matches.sessionB==sessionB_key) & (df_true_matches.idA==idA_key)].idB.values
                if len(idA_matches)<=0:
                    idA_matches = df_true_matches[(df_true_matches.sessionA==sessionB_key) & (df_true_matches.idB==idA_key)].idA.values
                    if len(idA_matches)<=0:
                        # print(f"Skipping {idA},{idB} as no matches for {idA_key} in {sessionB_key}")
                        continue
                idB_matches = df_true_matches[(df_true_matches.sessionA==sessionA_key) & (df_true_matches.idB==idB_key)].idA.values
                if len(idB_matches) <= 0:
                    idB_matches = df_true_matches[(df_true_matches.sessionB==sessionA_key) & (df_true_matches.idA==idB_key)].idB.values
                    if len(idB_matches) <= 0:
                        # print(f"Skipping {idA},{idB} as no matches for {idB_key} in {sessionA_key}")
                        continue
                
            
            gaze_embA, gaze_embB = course_input_dict[sessionA][idA]['gaze_emb'], course_input_dict[sessionB][idB]['gaze_emb']                
            clu_embA, clu_embB = course_input_dict[sessionA][idA]['cluster_emb'], course_input_dict[sessionB][idB]['cluster_emb']                
            
            if idA not in match_scores_gaze:
                match_scores_gaze[idA] = {}
            if idA not in match_scores_clu:
                match_scores_clu[idA] = {}
                
            if gaze_embA is None or gaze_embB is None:
                match_scores_gaze[idA][idB] = np.inf
            else:
                match_distance = cdist(gaze_embA.reshape(1,-1), gaze_embB.reshape(1,-1))[0][0]
                match_scores_gaze[idA][idB] = match_distance

            if clu_embA is None or clu_embB is None:
                match_scores_clu[idA][idB] = np.inf
            else:
                match_distance = cdist(clu_embA.reshape(1,-1), clu_embB.reshape(1,-1))[0][0]
                match_scores_clu[idA][idB] = match_distance
        
        df_match_gaze = pd.DataFrame(match_scores_gaze) 
        df_match_clu = pd.DataFrame(match_scores_clu) 
        gaze_cols = df_match_gaze.columns.values.tolist()
        clu_cols = df_match_clu.columns.values.tolist()
        all_cols = np.unique(gaze_cols+clu_cols)
        for col in all_cols:
            if col not in df_match_clu.columns:
                df_match_clu[col] = np.inf
            if col not in df_match_gaze.columns:
                df_match_clu[col] = np.inf
                
            sessionB_matches = deepcopy(df_match_clu[col]).sort_values().index.values.tolist() + \
                                deepcopy(df_match_gaze[col]).sort_values().index.values.tolist()
            if type=='unconstrained':
                sessionB_matches = deepcopy(df_match_clu[col]).sort_values().head(3).index.values.tolist() + \
                                deepcopy(df_match_gaze[col]).sort_values().head(3).index.values.tolist()
            sessionB_matches = np.unique(sessionB_matches)
            for match_id in sessionB_matches:
                if (match_scores_clu[col][match_id]<MATCH_THRESHOLD) | (match_scores_gaze[col][match_id]<MATCH_THRESHOLD):
                    col_face_area = course_input_dict[sessionA][col]['face_width_med'] * course_input_dict[sessionA][col]['face_height_med']
                    match_face_area = course_input_dict[sessionB][match_id]['face_width_med'] * course_input_dict[sessionB][match_id]['face_height_med']
                    rel_diff = np.abs(col_face_area-match_face_area)*100/min(col_face_area,match_face_area)
                    final_matches.append((sessionA_key, sessionB_key, f'{sessionA_key}_{col}', f'{sessionB_key}_{match_id}', match_scores_gaze[col][match_id], match_scores_clu[col][match_id], col_face_area, match_face_area))

df_final_matches = pd.DataFrame(final_matches, columns=['sessionA','sessionB','idA','idB','match_score_gaze','match_score_clu','face_areaA','face_areaB'])
df_final_matches.info()
df_final_matches.head(40)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606 entries, 0 to 605
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   sessionA          606 non-null    object 
 1   sessionB          606 non-null    object 
 2   idA               606 non-null    object 
 3   idB               606 non-null    object 
 4   match_score_gaze  606 non-null    float64
 5   match_score_clu   606 non-null    float64
 6   face_areaA        606 non-null    float32
 7   face_areaB        606 non-null    float32
dtypes: float32(2), float64(2), object(4)
memory usage: 33.3+ KB


,sessionA,sessionB,idA,idB,match_score_gaze,match_score_clu,face_areaA,face_areaB
0,0320,0410,0320_0,0410_0,0.284125,0.298760,2734.796143,4849.872070
1,0320,0410,0320_0,0410_1,0.265427,0.356003,2734.796143,3998.231201
2,0320,0410,0320_0,0410_2,0.293383,0.246724,2734.796143,4048.529053
3,0320,0410,0320_0,0410_3,0.240934,0.234685,2734.796143,3207.630371
4,0320,0410,0320_2,0410_0,0.190796,0.173122,3709.300781,4849.872070
5,0320,0410,0320_2,0410_1,0.233297,0.261140,3709.300781,3998.231201
6,0320,0410,0320_2,0410_2,0.205871,0.136874,3709.300781,4048.529053
7,0320,0410,0320_2,0410_3,0.273254,0.241818,3709.300781,3207.630371
8,0320,0410,0320_2,0410_4,0.339030,0.391650,3709.300781,2872.752441
9,0320,0410,0320_2,0410_7,0.387573,0.292734,3709.300781,3434.570068


In [370]:
final_course_ids = {}
session_pair_matches = {}
session_keys = df_final_matches.sessionA.unique()

for sessionA_idx in range(len(session_keys)):
    for sessionB_idx in range(sessionA_idx+1, len(session_keys)):
        # find all direct pairs
        
        direct_pairs = {}
        s1, s2 = session_keys[sessionA_idx], session_keys[sessionB_idx]
        df_pair_matches = df_final_matches[(df_final_matches.sessionA==s1) & (df_final_matches.sessionB==s2)]
        df_pair_matches = df_pair_matches.sort_values(by=['face_areaA','face_areaB'],ascending=False)
        df_pair_matches['area_diff'] = np.abs(df_pair_matches['face_areaA'] - df_pair_matches['face_areaB'])*100/np.minimum(df_pair_matches['face_areaA'],df_pair_matches['face_areaB'])
        df_pair_matches['avg_match_score'] = (df_pair_matches['match_score_gaze'] + df_pair_matches['match_score_clu'])/2
        # df_pair_matches['avg_match_score'] = df_pair_matches['match_score_gaze']
        # df_pair_matches['avg_match_score'] = df_pair_matches['match_score_clu']
        # get all eligible pairs with direct pairing
        matched_idBs = []
        for idA in df_pair_matches.idA.unique():
            idA_matches = df_pair_matches[(df_pair_matches.idA==idA) & (~(df_pair_matches.idB.isin(matched_idBs)))].sort_values(by='avg_match_score')[['idB','avg_match_score','area_diff']].values
            if idA_matches.shape[0] > 0:
                matched_idB, match_score, area_diff = idA_matches[0][0],idA_matches[0][1],idA_matches[0][2]
                matched_idBs.append(matched_idB)
                direct_pairs[(idA,matched_idB)] = (match_score, area_diff)

        inverse_pairs = {}
        s1, s2 = session_keys[sessionB_idx], session_keys[sessionA_idx]
        df_pair_matches = df_final_matches[(df_final_matches.sessionA==s1) & (df_final_matches.sessionB==s2)]
        df_pair_matches = df_pair_matches.sort_values(by=['face_areaA','face_areaB'],ascending=False)
        df_pair_matches['area_diff'] = np.abs(df_pair_matches['face_areaA'] - df_pair_matches['face_areaB'])*100/np.minimum(df_pair_matches['face_areaA'],df_pair_matches['face_areaB'])
        df_pair_matches['avg_match_score'] = (df_pair_matches['match_score_gaze'] + df_pair_matches['match_score_clu'])/2
        # df_pair_matches['avg_match_score'] = df_pair_matches['match_score_gaze']
        # df_pair_matches['avg_match_score'] = df_pair_matches['match_score_clu']
        # get all eligible pairs with direct pairing
        matched_idBs = []
        for idA in df_pair_matches.idA.unique():
            idA_matches = df_pair_matches[(df_pair_matches.idA==idA) & (~(df_pair_matches.idB.isin(matched_idBs)))].sort_values(by='avg_match_score')[['idB','avg_match_score','area_diff']].values
            if idA_matches.shape[0] > 0:
                matched_idB, match_score, area_diff = idA_matches[0][0],idA_matches[0][1],idA_matches[0][2]
                matched_idBs.append(matched_idB)
                inverse_pairs[(matched_idB,idA)] = (match_score, area_diff)

        final_pairs = []
        paired_idAs, paired_idBs = [],[]
        for pair_key in direct_pairs:
            if pair_key in inverse_pairs:
                if (pair_key[0] not in paired_idAs) & (pair_key[0] not in paired_idBs):
                    final_pairs.append((pair_key, direct_pairs[pair_key],inverse_pairs[pair_key]))
                    paired_idAs.append(pair_key[0])
                    paired_idBs.append(pair_key[1])
            elif (direct_pairs[pair_key][0] < MATCH_PAIR_MAX_THRESHOLD) & (direct_pairs[pair_key][1] < MATCH_PAIR_AREA_THRESHOLD):
                if (pair_key[0] not in paired_idAs) & (pair_key[0] not in paired_idBs):
                    final_pairs.append((pair_key, direct_pairs[pair_key],-1))
                    paired_idAs.append(pair_key[0])
                    paired_idBs.append(pair_key[1])
        
        for pair_key in inverse_pairs:
            if (pair_key not in direct_pairs) & (inverse_pairs[pair_key][0] < MATCH_PAIR_MAX_THRESHOLD) & (inverse_pairs[pair_key][1] < MATCH_PAIR_AREA_THRESHOLD):
                if (pair_key[0] not in paired_idAs) & (pair_key[0] not in paired_idBs):
                    final_pairs.append((pair_key, -1,inverse_pairs[pair_key]))
                    paired_idAs.append(pair_key[0])
                    paired_idBs.append(pair_key[1])
        session_pair_matches[(session_keys[sessionA_idx], session_keys[sessionB_idx])] = final_pairs


        
        


In [371]:
all_id_sets = []
for session_pair in session_pair_matches:
    id_pairs_info = session_pair_matches[session_pair]
    for (id_pair, direct_match_score, inv_match_score) in id_pairs_info:
        found_pair = False
        for id_set in all_id_sets:
            if (id_pair[0] in id_set) or (id_pair[1] in id_set):
                found_pair =  True
                id_set.add(id_pair[0])
                id_set.add(id_pair[1])
                break
        # print(all_id_sets, id_pair, found_pair)        
        if not found_pair:
            new_set = set()
            new_set.add(id_pair[0])
            new_set.add(id_pair[1])
            all_id_sets.append(new_set)


In [372]:
id_pair = ('0424_5', '0501_11')

In [373]:
def id_pair_in_gt(df_gt, id_pair):
    s1, id1 = id_pair[0].split("_")
    s2, id2 = id_pair[1].split("_")
    for gt_idx, gt_row in df_gt.iterrows():
        s1_matches = list(map(float,str(gt_row[f'Session-{s1}']).split(",")))
        s2_matches = list(map(float,str(gt_row[f'Session-{s2}']).split(",")))
        if (float(id1) in s1_matches) & (float(id2) in s2_matches):
            return True
    return False

In [374]:
num_true_pairs, num_total_pairs = 0,0
predicted_pairs = []
for session_pair in session_pair_matches:
    id_pairs_info = session_pair_matches[session_pair]
    for (id_pair, direct_match_score, inv_match_score) in id_pairs_info:
        if session_cols.index(f'Session-{id_pair[0].split("_")[0]}') < session_cols.index(f'Session-{id_pair[1].split("_")[0]}'):
            predicted_pairs.append([id_pair[0],id_pair[1]])
        else:
            predicted_pairs.append([id_pair[1],id_pair[0]])
        if id_pair_in_gt(df_gt, id_pair):
            # print("True Pair", (id_pair, direct_match_score, inv_match_score))
            print(id_pair)
            num_true_pairs+=1
        else:
            ...
            # print("False Pair", (id_pair, direct_match_score, inv_match_score))
        num_total_pairs+=1
df_predicted = pd.DataFrame(predicted_pairs, columns=['idA','idB'])
df_predicted['pred']=1
# df_predicted

('0320_8', '0410_2')
('0320_4', '0410_4')
('0320_8', '0417_13')
('0320_4', '0417_2')
('0320_8', '0424_13')
('0320_9', '0424_9')
('0320_2', '0424_1')
('0320_4', '0424_11')
('0320_8', '0501_2')
('0320_4', '0501_6')
('0410_0', '0417_3')
('0410_6', '0417_1')
('0410_2', '0417_13')
('0410_4', '0417_2')
('0410_13', '0417_14')
('0410_0', '0424_1')
('0410_0', '0501_5')
('0417_13', '0424_13')
('0417_13', '0501_2')
('0417_0', '0501_3')
('0417_2', '0501_6')
('0417_8', '0501_10')
('0424_6', '0501_13')
('0424_1', '0501_5')
('0424_13', '0501_2')
('0424_11', '0501_6')
('0424_2', '0501_10')


In [375]:
df_true_matches[df_true_matches.idA=='0410_2']

,sessionA,sessionB,idA,idB,gt
1,0410,0417,0410_2,0417_13,1
7,0410,0424,0410_2,0424_13,1
13,0410,0501,0410_2,0501_2,1
20,0410,0320,0410_2,0320_8,1


In [376]:
session_cols

['Session-0410',
 'Session-0417',
 'Session-0424',
 'Session-0501',
 'Session-0320']

In [377]:
num_true_pairs, num_total_pairs

(27, 30)

In [378]:
df_all_matches = pd.merge(df_true_matches, df_predicted,on=['idA','idB'], how='outer')
df_all_matches.loc[df_all_matches['gt'].isnull(),'gt'] = 0
df_all_matches.loc[df_all_matches['pred'].isnull(),'pred'] = 0
df_all_matches

,sessionA,sessionB,idA,idB,gt,pred
0,0410,0417,0410_0,0417_3,1.0,1.0
1,0410,0417,0410_2,0417_13,1.0,1.0
2,0410,0417,0410_4,0417_2,1.0,1.0
3,0410,0417,0410_6,0417_1,1.0,1.0
4,0410,0417,0410_7,0417_5,1.0,0.0
...,...,...,...,...,...,...
63,0501,0320,0501_4,0320_1,1.0,0.0
64,0501,0320,0501_3,0320_10,1.0,0.0
65,NaN,NaN,0501_5,0320_9,0.0,1.0
66,NaN,NaN,0501_3,0320_2,0.0,1.0


In [379]:
df_all_matches.to_csv(f"case_studies/results/across_session_{type}_matches_{course}.csv", index=False)

In [380]:
df_all_matches[(df_all_matches.pred==1) & (df_all_matches['gt']==1)].shape, df_all_matches.shape

((27, 6), (68, 6))

In [381]:
print(precision_score(df_all_matches['gt'], df_all_matches['pred'], average=None), recall_score(df_all_matches['gt'], df_all_matches['pred'], average=None))

[0.  0.9] [0.         0.41538462]


In [382]:
precision_score(df_all_matches['gt'], df_all_matches['pred']), recall_score(df_all_matches['gt'], df_all_matches['pred'])

(0.9, 0.4153846153846154)

# Hyperparameter optimization

In [156]:
threshold_optimization = []
match_threshold_choices = np.arange(0.15,1,0.05)
match_pair_max_threshold_choices = np.arange(0,0.5,0.05)
match_pair_area_threshold_choices = np.arange(0,100,5)
hyperparameter_results =[] 

for (MATCH_THRESHOLD, MATCH_PAIR_MAX_THRESHOLD, MATCH_PAIR_AREA_THRESHOLD) in product(match_threshold_choices, match_pair_max_threshold_choices, match_pair_area_threshold_choices):
    if MATCH_THRESHOLD<MATCH_PAIR_MAX_THRESHOLD:
        continue
    final_matches = []
    for (sessionA, sessionB) in product(sorted(sessions), sorted(sessions)):
        sessionA_key, sessionB_key = sessionA.split("_")[-1][4:8], sessionB.split("_")[-1][4:8]
        if not (sessionA==sessionB):
            # if sessionA not in session_matches:
            # match session A and session B based on gaze clustering
            match_scores_gaze = {}
            match_scores_clu  = {}
            for idA,idB in product(course_input_dict[sessionA].keys(), course_input_dict[sessionB].keys()):
                # check if there exists a gt match for idA in SessionB
                # idA_key = f'{sessionA_key}_{idA}'
                # idB_key = f'{sessionB_key}_{idB}'
                # idA_matches = df_true_matches[(df_true_matches.sessionB==sessionB_key) & (df_true_matches.idA==idA_key)].idB.values
                # if len(idA_matches)<=0:
                #     idA_matches = df_true_matches[(df_true_matches.sessionA==sessionB_key) & (df_true_matches.idB==idA_key)].idA.values
                #     if len(idA_matches)<=0:
                #         # print(f"Skipping {idA},{idB} as no matches for {idA_key} in {sessionB_key}")
                #         continue
                # idB_matches = df_true_matches[(df_true_matches.sessionA==sessionA_key) & (df_true_matches.idB==idB_key)].idA.values
                # if len(idB_matches) <= 0:
                #     idB_matches = df_true_matches[(df_true_matches.sessionB==sessionA_key) & (df_true_matches.idA==idB_key)].idB.values
                #     if len(idB_matches) <= 0:
                #         # print(f"Skipping {idA},{idB} as no matches for {idB_key} in {sessionA_key}")
                #         continue

                gaze_embA, gaze_embB = course_input_dict[sessionA][idA]['gaze_emb'], course_input_dict[sessionB][idB]['gaze_emb']                
                clu_embA, clu_embB = course_input_dict[sessionA][idA]['cluster_emb'], course_input_dict[sessionB][idB]['cluster_emb']                
                
                if idA not in match_scores_gaze:
                    match_scores_gaze[idA] = {}
                if idA not in match_scores_clu:
                    match_scores_clu[idA] = {}
                    
                if gaze_embA is None or gaze_embB is None:
                    match_scores_gaze[idA][idB] = np.inf
                else:
                    match_distance = cdist(gaze_embA.reshape(1,-1), gaze_embB.reshape(1,-1))[0][0]
                    match_scores_gaze[idA][idB] = match_distance
    
                if clu_embA is None or clu_embB is None:
                    match_scores_clu[idA][idB] = np.inf
                else:
                    match_distance = cdist(clu_embA.reshape(1,-1), clu_embB.reshape(1,-1))[0][0]
                    match_scores_clu[idA][idB] = match_distance
            
            df_match_gaze = pd.DataFrame(match_scores_gaze) 
            df_match_clu = pd.DataFrame(match_scores_clu) 
            gaze_cols = df_match_gaze.columns.values.tolist()
            clu_cols = df_match_clu.columns.values.tolist()
            all_cols = np.unique(gaze_cols+clu_cols)
            for col in all_cols:
                if col not in df_match_clu.columns:
                    df_match_clu[col] = np.inf
                if col not in df_match_gaze.columns:
                    df_match_clu[col] = np.inf
                sessionB_matches = deepcopy(df_match_clu[col]).sort_values().head(3).index.values.tolist() + \
                                    deepcopy(df_match_gaze[col]).sort_values().head(3).index.values.tolist()
                sessionB_matches = np.unique(sessionB_matches)
                for match_id in sessionB_matches:
                    if (match_scores_clu[col][match_id]<MATCH_THRESHOLD) | (match_scores_gaze[col][match_id]<MATCH_THRESHOLD):
                        col_face_area = course_input_dict[sessionA][col]['face_width_med'] * course_input_dict[sessionA][col]['face_height_med']
                        match_face_area = course_input_dict[sessionB][match_id]['face_width_med'] * course_input_dict[sessionB][match_id]['face_height_med']
                        rel_diff = np.abs(col_face_area-match_face_area)*100/min(col_face_area,match_face_area)
                        final_matches.append((sessionA_key, sessionB_key, f'{sessionA_key}_{col}', f'{sessionB_key}_{match_id}', match_scores_gaze[col][match_id], match_scores_clu[col][match_id], col_face_area, match_face_area))
    
    df_final_matches = pd.DataFrame(final_matches, columns=['sessionA','sessionB','idA','idB','match_score_gaze','match_score_clu','face_areaA','face_areaB'])
    
    
    final_course_ids = {}
    session_pair_matches = {}
    session_keys = df_final_matches.sessionA.unique()
    
    for sessionA_idx in range(len(session_keys)):
        for sessionB_idx in range(sessionA_idx+1, len(session_keys)):
            # find all direct pairs
            
            direct_pairs = {}
            s1, s2 = session_keys[sessionA_idx], session_keys[sessionB_idx]
            df_pair_matches = df_final_matches[(df_final_matches.sessionA==s1) & (df_final_matches.sessionB==s2)]
            df_pair_matches = df_pair_matches.sort_values(by=['face_areaA','face_areaB'],ascending=False)
            df_pair_matches['area_diff'] = np.abs(df_pair_matches['face_areaA'] - df_pair_matches['face_areaB'])*100/np.minimum(df_pair_matches['face_areaA'],df_pair_matches['face_areaB'])
            df_pair_matches['avg_match_score'] = (df_pair_matches['match_score_gaze'] + df_pair_matches['match_score_clu'])/2
            # df_pair_matches['avg_match_score'] = df_pair_matches['match_score_gaze']
            # df_pair_matches['avg_match_score'] = df_pair_matches['match_score_clu']
            # get all eligible pairs with direct pairing
            matched_idBs = []
            for idA in df_pair_matches.idA.unique():
                idA_matches = df_pair_matches[(df_pair_matches.idA==idA) & (~(df_pair_matches.idB.isin(matched_idBs)))].sort_values(by='avg_match_score')[['idB','avg_match_score','area_diff']].values
                if idA_matches.shape[0] > 0:
                    matched_idB, match_score, area_diff = idA_matches[0][0],idA_matches[0][1],idA_matches[0][2]
                    matched_idBs.append(matched_idB)
                    direct_pairs[(idA,matched_idB)] = (match_score, area_diff)
    
            inverse_pairs = {}
            s1, s2 = session_keys[sessionB_idx], session_keys[sessionA_idx]
            df_pair_matches = df_final_matches[(df_final_matches.sessionA==s1) & (df_final_matches.sessionB==s2)]
            df_pair_matches = df_pair_matches.sort_values(by=['face_areaA','face_areaB'],ascending=False)
            df_pair_matches['area_diff'] = np.abs(df_pair_matches['face_areaA'] - df_pair_matches['face_areaB'])*100/np.minimum(df_pair_matches['face_areaA'],df_pair_matches['face_areaB'])
            df_pair_matches['avg_match_score'] = (df_pair_matches['match_score_gaze'] + df_pair_matches['match_score_clu'])/2
            # df_pair_matches['avg_match_score'] = df_pair_matches['match_score_gaze']
            # df_pair_matches['avg_match_score'] = df_pair_matches['match_score_clu']
            # get all eligible pairs with direct pairing
            matched_idBs = []
            for idA in df_pair_matches.idA.unique():
                idA_matches = df_pair_matches[(df_pair_matches.idA==idA) & (~(df_pair_matches.idB.isin(matched_idBs)))].sort_values(by='avg_match_score')[['idB','avg_match_score','area_diff']].values
                if idA_matches.shape[0] > 0:
                    matched_idB, match_score, area_diff = idA_matches[0][0],idA_matches[0][1],idA_matches[0][2]
                    matched_idBs.append(matched_idB)
                    inverse_pairs[(matched_idB,idA)] = (match_score, area_diff)
    
            final_pairs = []
            paired_idAs, paired_idBs = [],[]
            for pair_key in direct_pairs:
                if pair_key in inverse_pairs:
                    if (pair_key[0] not in paired_idAs) & (pair_key[0] not in paired_idBs):
                        final_pairs.append((pair_key, direct_pairs[pair_key],inverse_pairs[pair_key]))
                        paired_idAs.append(pair_key[0])
                        paired_idBs.append(pair_key[1])
                elif (direct_pairs[pair_key][0] < MATCH_PAIR_MAX_THRESHOLD) & (direct_pairs[pair_key][1] < MATCH_PAIR_AREA_THRESHOLD):
                    if (pair_key[0] not in paired_idAs) & (pair_key[0] not in paired_idBs):
                        final_pairs.append((pair_key, direct_pairs[pair_key],-1))
                        paired_idAs.append(pair_key[0])
                        paired_idBs.append(pair_key[1])
            
            for pair_key in inverse_pairs:
                if (pair_key not in direct_pairs) & (inverse_pairs[pair_key][0] < MATCH_PAIR_MAX_THRESHOLD) & (inverse_pairs[pair_key][1] < MATCH_PAIR_AREA_THRESHOLD):
                    if (pair_key[0] not in paired_idAs) & (pair_key[0] not in paired_idBs):
                        final_pairs.append((pair_key, -1,inverse_pairs[pair_key]))
                        paired_idAs.append(pair_key[0])
                        paired_idBs.append(pair_key[1])
            session_pair_matches[(session_keys[sessionA_idx], session_keys[sessionB_idx])] = final_pairs
    
    all_id_sets = []
    for session_pair in session_pair_matches:
        id_pairs_info = session_pair_matches[session_pair]
        for (id_pair, direct_match_score, inv_match_score) in id_pairs_info:
            found_pair = False
            for id_set in all_id_sets:
                if (id_pair[0] in id_set) or (id_pair[1] in id_set):
                    found_pair =  True
                    id_set.add(id_pair[0])
                    id_set.add(id_pair[1])
                    break
            # print(all_id_sets, id_pair, found_pair)        
            if not found_pair:
                new_set = set()
                new_set.add(id_pair[0])
                new_set.add(id_pair[1])
                all_id_sets.append(new_set)
    
    num_true_pairs, num_total_pairs = 0,0
    for session_pair in session_pair_matches:
        id_pairs_info = session_pair_matches[session_pair]
        for (id_pair, direct_match_score, inv_match_score) in id_pairs_info:
            if id_pair_in_gt(df_gt, id_pair):
                # print("True Pair", (id_pair, direct_match_score, inv_match_score))
                num_true_pairs+=1
            # else:
            num_total_pairs+=1
                # print("False Pair", (id_pair, direct_match_score, inv_match_score))
                
            
            
    if num_total_pairs > 0:
        hyperparameter_results.append([MATCH_THRESHOLD, MATCH_PAIR_MAX_THRESHOLD, MATCH_PAIR_AREA_THRESHOLD, num_true_pairs, num_total_pairs, num_true_pairs/num_total_pairs])
        print(MATCH_THRESHOLD, MATCH_PAIR_MAX_THRESHOLD, MATCH_PAIR_AREA_THRESHOLD, num_true_pairs, num_total_pairs, num_true_pairs/num_total_pairs)
    else:
        print(MATCH_THRESHOLD, MATCH_PAIR_MAX_THRESHOLD, MATCH_PAIR_AREA_THRESHOLD, num_true_pairs, num_total_pairs)


0.15 0.0 0 10 16 0.625
0.15 0.0 5 10 16 0.625
0.15 0.0 10 10 16 0.625
0.15 0.0 15 10 16 0.625
0.15 0.0 20 10 16 0.625
0.15 0.0 25 10 16 0.625
0.15 0.0 30 10 16 0.625
0.15 0.0 35 10 16 0.625
0.15 0.0 40 10 16 0.625
0.15 0.0 45 10 16 0.625
0.15 0.0 50 10 16 0.625
0.15 0.0 55 10 16 0.625
0.15 0.0 60 10 16 0.625
0.15 0.0 65 10 16 0.625
0.15 0.0 70 10 16 0.625
0.15 0.0 75 10 16 0.625
0.15 0.0 80 10 16 0.625
0.15 0.0 85 10 16 0.625
0.15 0.0 90 10 16 0.625
0.15 0.0 95 10 16 0.625
0.15 0.05 0 10 16 0.625
0.15 0.05 5 10 16 0.625
0.15 0.05 10 10 16 0.625
0.15 0.05 15 10 16 0.625
0.15 0.05 20 10 16 0.625
0.15 0.05 25 10 16 0.625
0.15 0.05 30 10 16 0.625
0.15 0.05 35 10 16 0.625
0.15 0.05 40 10 16 0.625
0.15 0.05 45 10 16 0.625
0.15 0.05 50 10 16 0.625
0.15 0.05 55 10 16 0.625
0.15 0.05 60 10 16 0.625
0.15 0.05 65 10 16 0.625
0.15 0.05 70 10 16 0.625
0.15 0.05 75 10 16 0.625
0.15 0.05 80 10 16 0.625
0.15 0.05 85 10 16 0.625
0.15 0.05 90 10 16 0.625
0.15 0.05 95 10 16 0.625
0.15 0.1 0 10 16 0.625
0


KeyboardInterrupt



In [157]:
df_results = pd.DataFrame(hyperparameter_results, columns=['max_thr','max_pair_thr','max_area_thr','num_true_pairs','num_total_pairs','precision'])
df_results.shape

(1076, 6)

In [160]:
df_results[df_results.num_total_pairs>=25].sort_values(by='precision',ascending=False).head(20)

,max_thr,max_pair_thr,max_area_thr,num_true_pairs,num_total_pairs,precision
504,0.35,0.2,20,29,48,0.604167
507,0.35,0.2,35,30,50,0.600000
505,0.35,0.2,25,30,50,0.600000
508,0.35,0.2,40,30,50,0.600000
506,0.35,0.2,30,30,50,0.600000
664,0.40,0.2,20,28,47,0.595745
668,0.40,0.2,40,29,49,0.591837
667,0.40,0.2,35,29,49,0.591837
666,0.40,0.2,30,29,49,0.591837
665,0.40,0.2,25,29,49,0.591837


In [353]:
# s1,s2 = '0228','0205'
# df_pair_matches = df_final_matches[(df_final_matches.sessionA==s1) & (df_final_matches.sessionB==s2)]
# df_pair_matches

In [354]:
# df_pair_matches = df_pair_matches.sort_values(by=['face_areaA','face_areaB'],ascending=False)
# df_pair_matches['area_diff'] = np.abs(df_pair_matches['face_areaA'] - df_pair_matches['face_areaB'])*100/np.minimum(df_pair_matches['face_areaA'],df_pair_matches['face_areaB'])
# df_pair_matches['avg_match_score'] = (df_pair_matches['match_score_gaze'] + df_pair_matches['match_score_clu'])/2
# df_pair_matches[['idA','idB','area_diff','avg_match_score']]
# # df_pair_matches.sort_values(by='idA')

In [355]:
# df_pair_matches.idA.unique()

In [356]:
# get final pairs based on df_pair_matches
# final_pairs = {}
# matched_idBs = []
# pair_match_max=0.2
# for idA in df_pair_matches.idA.unique():
#     idA_matches = df_pair_matches[(df_pair_matches.idA==idA) & (~(df_pair_matches.idB.isin(matched_idBs)))].sort_values(by='avg_match_score')[['idB','avg_match_score']].values
#     if idA_matches.shape[0] > 0:
#         matched_idB, match_score = idA_matches[0][0],idA_matches[0][1]
#         if match_score<pair_match_max:
#             matched_idBs.append(matched_idB)
#             print(idA,matched_idB,match_score)  
#         else:
#             #potential pair
#             print("Potential Pair:", idA,matched_idB,match_score)  

In [357]:
# get final pairs based on df_pair_matches
# final_pairs = {}
# matched_idBs = []
# # pair_match_max=0.2
# for idA in df_pair_matches.idA.unique():
#     idA_matches = df_pair_matches[(df_pair_matches.idA==idA) & (~(df_pair_matches.idB.isin(matched_idBs)))].sort_values(by='avg_match_score')[['idB','avg_match_score']].values
#     if idA_matches.shape[0] > 0:
#         matched_idB, match_score = idA_matches[0][0],idA_matches[0][1]
#         matched_idBs.append(matched_idB)
#         print(idA,matched_idB,match_score)

In [194]:
df_pair_matches[(df_pair_matches.match_score_gaze<0.2) & (df_pair_matches.match_score_clu<0.2) & (df_pair_matches.area_diff<20) & (np.minimum(df_pair_matches.face_areaA, df_pair_matches.face_areaB)>np.median(df_pair_matches.face_areaA))]

,sessionA,sessionB,idA,idB,match_score_gaze,match_score_clu,face_areaA,face_areaB,area_diff


In [195]:
id_matches = {}
matched_idA, matched_idB = [],[]

best_match_1 = df_pair_matches[(df_pair_matches.match_score_gaze<0.2) & 
                                (df_pair_matches.match_score_clu<0.2) & 
                                (df_pair_matches.area_diff<20) & 
                                (np.minimum(df_pair_matches.face_areaA, df_pair_matches.face_areaB)>np.median(df_pair_matches.face_areaA))
                                ].sort_values(by='match_score_clu')
sprint(best_match_1)
for idx,row in best_match_1.iterrows():
    if row['idA'] in matched_idA:
        continue
    elif row['idB'] in matched_idB:
        continue
    else:
        id_matches[row['idA']] = row['idB']
        matched_idA.append(row['idA'])
        matched_idB.append(row['idB'])
df_next_matches = df_pair_matches[(~df_pair_matches.idA.isin(matched_idA)) & (~df_pair_matches.idB.isin(matched_idB))]
id_matches

best_match_1 : Empty DataFrame
Columns: [sessionA, sessionB, idA, idB, match_score_gaze, match_score_clu, face_areaA, face_areaB, area_diff]
Index: []


{}

In [196]:
df_next_matches = pd.merge(df_next_matches, df_next_matches.groupby('idA',as_index=False)['idB'].count(), on='idA',suffixes=('','_count'))
df_next_matches

,sessionA,sessionB,idA,idB,match_score_gaze,match_score_clu,face_areaA,face_areaB,area_diff,idB_count
0,0227,0306,0227_11,0306_2,0.282416,0.331963,7133.739258,5449.581055,30.904360,1
1,0227,0306,0227_1,0306_9,0.201522,0.217121,5123.483398,3704.526855,38.303314,1
2,0227,0306,0227_9,0306_5,0.282521,0.270811,5078.155762,4444.170410,14.265550,3
3,0227,0306,0227_9,0306_10,0.406093,0.281338,5078.155762,3421.178955,48.432919,3
4,0227,0306,0227_9,0306_0,0.322274,0.279022,5078.155762,2632.532471,92.900024,3
5,0227,0306,0227_5,0306_2,0.338447,0.299433,4878.859863,5449.581055,11.697839,1
6,0227,0306,0227_6,0306_5,0.309760,0.271243,4873.996582,4444.170410,9.671685,2
7,0227,0306,0227_6,0306_10,0.348805,0.297352,4873.996582,3421.178955,42.465408,2
8,0227,0306,0227_8,0306_2,0.290177,0.252726,4041.062500,5449.581055,34.855156,1
9,0227,0306,0227_0,0306_9,0.259295,0.212097,3476.407715,3704.526855,6.561922,2


In [197]:
df_single_matches = df_next_matches[df_next_matches.idB_count<=1]
best_match_2 = df_single_matches[(df_single_matches.match_score_gaze<0.25) & 
                                (df_single_matches.match_score_clu<0.25) & 
                                # (df_single_matches.area_diff<20) & 
                                (np.minimum(df_single_matches.face_areaA, df_single_matches.face_areaB)>np.median(df_pair_matches.face_areaA))
                                ].sort_values(by='match_score_gaze')
sprint(best_match_2)
for idx,row in best_match_2.iterrows():
    if row['idA'] in matched_idA:
        continue
    elif row['idB'] in matched_idB:
        continue
    else:
        id_matches[row['idA']] = row['idB']
        matched_idA.append(row['idA'])
        matched_idB.append(row['idB'])
df_next_matches = df_pair_matches[(~df_pair_matches.idA.isin(matched_idA)) & (~df_pair_matches.idB.isin(matched_idB))]
id_matches

best_match_2 :   sessionA sessionB     idA     idB  match_score_gaze  match_score_clu  \
1     0227     0306  0227_1  0306_9          0.201522         0.217121   

    face_areaA   face_areaB  area_diff  idB_count  
1  5123.483398  3704.526855  38.303314          1  


{'0227_1': '0306_9'}

In [198]:
df_next_matches = pd.merge(df_next_matches, df_next_matches.groupby('idB',as_index=False)['idA'].count(), on='idB',suffixes=('','_count'))
df_next_matches

,sessionA,sessionB,idA,idB,match_score_gaze,match_score_clu,face_areaA,face_areaB,area_diff,idA_count
0,0227,0306,0227_11,0306_2,0.282416,0.331963,7133.739258,5449.581055,30.904360,3
1,0227,0306,0227_5,0306_2,0.338447,0.299433,4878.859863,5449.581055,11.697839,3
2,0227,0306,0227_8,0306_2,0.290177,0.252726,4041.062500,5449.581055,34.855156,3
3,0227,0306,0227_9,0306_5,0.282521,0.270811,5078.155762,4444.170410,14.265550,4
4,0227,0306,0227_6,0306_5,0.309760,0.271243,4873.996582,4444.170410,9.671685,4
5,0227,0306,0227_3,0306_5,0.276308,0.239300,2946.561523,4444.170410,50.825645,4
6,0227,0306,0227_2,0306_5,0.332398,0.290164,2886.754639,4444.170410,53.950405,4
7,0227,0306,0227_9,0306_10,0.406093,0.281338,5078.155762,3421.178955,48.432919,4
8,0227,0306,0227_6,0306_10,0.348805,0.297352,4873.996582,3421.178955,42.465408,4
9,0227,0306,0227_3,0306_10,0.306611,0.207327,2946.561523,3421.178955,16.107500,4


In [199]:
best_match_3 = df_next_matches[(df_next_matches.idA_count<=1) & 
                                (df_next_matches.area_diff<20)].sort_values(by='match_score_gaze')
sprint(best_match_3)
for idx,row in best_match_3.iterrows():
    if row['idA'] in matched_idA:
        continue
    elif row['idB'] in matched_idB:
        continue
    else:
        id_matches[row['idA']] = row['idB']
        matched_idA.append(row['idA'])
        matched_idB.append(row['idB'])

best_match_3 :    sessionA sessionB     idA     idB  match_score_gaze  match_score_clu  \
17     0227     0306  0227_7  0306_4           0.27258         0.340231   

     face_areaA   face_areaB  area_diff  idA_count  
17  2819.162109  2558.156006    10.2029          1  


In [200]:
df_next_matches = df_pair_matches[(~df_pair_matches.idA.isin(matched_idA)) & (~df_pair_matches.idB.isin(matched_idB))]
id_matches

{'0227_1': '0306_9', '0227_7': '0306_4'}

In [201]:
df_next_matches = pd.merge(df_next_matches, df_next_matches.groupby('idB',as_index=False)['idA'].count(), on='idB',suffixes=('','_count'))
df_next_matches = pd.merge(df_next_matches, df_next_matches.groupby('idA',as_index=False)['idB'].count(), on='idA',suffixes=('','_count'))
df_next_matches.sort_values(by='idA_count')

,sessionA,sessionB,idA,idB,match_score_gaze,match_score_clu,face_areaA,face_areaB,area_diff,idA_count,idB_count
10,0227,0306,0227_3,0306_1,0.105268,0.139410,2946.561523,2716.470215,8.470231,2,3
14,0227,0306,0227_2,0306_1,0.284506,0.253402,2886.754639,2716.470215,6.268591,2,4
0,0227,0306,0227_11,0306_2,0.282416,0.331963,7133.739258,5449.581055,30.904360,3,1
1,0227,0306,0227_5,0306_2,0.338447,0.299433,4878.859863,5449.581055,11.697839,3,1
2,0227,0306,0227_8,0306_2,0.290177,0.252726,4041.062500,5449.581055,34.855156,3,1
5,0227,0306,0227_9,0306_0,0.322274,0.279022,5078.155762,2632.532471,92.900024,3,3
13,0227,0306,0227_2,0306_0,0.312086,0.296234,2886.754639,2632.532471,9.656943,3,4
15,0227,0306,0227_0,0306_0,0.293580,0.281686,3476.407715,2632.532471,32.055645,3,1
3,0227,0306,0227_9,0306_5,0.282521,0.270811,5078.155762,4444.170410,14.265550,4,3
4,0227,0306,0227_9,0306_10,0.406093,0.281338,5078.155762,3421.178955,48.432919,4,3


In [202]:
df_next_matches = pd.merge(df_next_matches, df_next_matches.groupby('idA',as_index=False).agg({'match_score_clu':lambda x: 1.0 if (len(x)<2) else sorted(x)[1]-sorted(x)[0]}), on='idA',suffixes=('','_best_diff'))
df_next_matches

,sessionA,sessionB,idA,idB,match_score_gaze,match_score_clu,face_areaA,face_areaB,area_diff,idA_count,idB_count,match_score_clu_best_diff
0,0227,0306,0227_11,0306_2,0.282416,0.331963,7133.739258,5449.581055,30.904360,3,1,1.000000
1,0227,0306,0227_5,0306_2,0.338447,0.299433,4878.859863,5449.581055,11.697839,3,1,1.000000
2,0227,0306,0227_8,0306_2,0.290177,0.252726,4041.062500,5449.581055,34.855156,3,1,1.000000
3,0227,0306,0227_9,0306_5,0.282521,0.270811,5078.155762,4444.170410,14.265550,4,3,0.008210
4,0227,0306,0227_9,0306_10,0.406093,0.281338,5078.155762,3421.178955,48.432919,4,3,0.008210
5,0227,0306,0227_9,0306_0,0.322274,0.279022,5078.155762,2632.532471,92.900024,3,3,0.008210
6,0227,0306,0227_6,0306_5,0.309760,0.271243,4873.996582,4444.170410,9.671685,4,2,0.026109
7,0227,0306,0227_6,0306_10,0.348805,0.297352,4873.996582,3421.178955,42.465408,4,2,0.026109
8,0227,0306,0227_3,0306_5,0.276308,0.239300,2946.561523,4444.170410,50.825645,4,3,0.067917
9,0227,0306,0227_3,0306_10,0.306611,0.207327,2946.561523,3421.178955,16.107500,4,3,0.067917


In [203]:
df_next_matches = pd.merge(df_next_matches, df_next_matches.groupby('idA',as_index=False).agg({'match_score_clu':lambda x: sorted(x)[0]}), on='idA',suffixes=('','_min_match'))
df_next_matches

,sessionA,sessionB,idA,idB,match_score_gaze,match_score_clu,face_areaA,face_areaB,area_diff,idA_count,idB_count,match_score_clu_best_diff,match_score_clu_min_match
0,0227,0306,0227_11,0306_2,0.282416,0.331963,7133.739258,5449.581055,30.904360,3,1,1.000000,0.331963
1,0227,0306,0227_5,0306_2,0.338447,0.299433,4878.859863,5449.581055,11.697839,3,1,1.000000,0.299433
2,0227,0306,0227_8,0306_2,0.290177,0.252726,4041.062500,5449.581055,34.855156,3,1,1.000000,0.252726
3,0227,0306,0227_9,0306_5,0.282521,0.270811,5078.155762,4444.170410,14.265550,4,3,0.008210,0.270811
4,0227,0306,0227_9,0306_10,0.406093,0.281338,5078.155762,3421.178955,48.432919,4,3,0.008210,0.270811
5,0227,0306,0227_9,0306_0,0.322274,0.279022,5078.155762,2632.532471,92.900024,3,3,0.008210,0.270811
6,0227,0306,0227_6,0306_5,0.309760,0.271243,4873.996582,4444.170410,9.671685,4,2,0.026109,0.271243
7,0227,0306,0227_6,0306_10,0.348805,0.297352,4873.996582,3421.178955,42.465408,4,2,0.026109,0.271243
8,0227,0306,0227_3,0306_5,0.276308,0.239300,2946.561523,4444.170410,50.825645,4,3,0.067917,0.139410
9,0227,0306,0227_3,0306_10,0.306611,0.207327,2946.561523,3421.178955,16.107500,4,3,0.067917,0.139410


In [204]:
best_match_4 = df_next_matches[(df_next_matches.match_score_gaze<0.25) &
                (df_next_matches.match_score_clu_best_diff>0.05) &
                (df_next_matches.match_score_clu==df_next_matches.match_score_clu_min_match)].sort_values(by='match_score_clu')
sprint(best_match_4)
for idx,row in best_match_4.iterrows():
    if row['idA'] in matched_idA:
        continue
    elif row['idB'] in matched_idB:
        continue
    else:
        id_matches[row['idA']] = row['idB']
        matched_idA.append(row['idA'])
        matched_idB.append(row['idB'])
        


best_match_4 :    sessionA sessionB     idA     idB  match_score_gaze  match_score_clu  \
10     0227     0306  0227_3  0306_1          0.105268          0.13941   

     face_areaA   face_areaB  area_diff  idA_count  idB_count  \
10  2946.561523  2716.470215   8.470231          2          3   

    match_score_clu_best_diff  match_score_clu_min_match  
10                   0.067917                    0.13941  


In [205]:
df_next_matches = df_pair_matches[(~df_pair_matches.idA.isin(matched_idA)) & (~df_pair_matches.idB.isin(matched_idB))]
id_matches

{'0227_1': '0306_9', '0227_7': '0306_4', '0227_3': '0306_1'}

In [206]:
df_next_matches = df_next_matches.sort_values(by=['face_areaA'],ascending=False)
df_next_matches

,sessionA,sessionB,idA,idB,match_score_gaze,match_score_clu,face_areaA,face_areaB,area_diff
21,0227,0306,0227_11,0306_2,0.282416,0.331963,7133.739258,5449.581055,30.904360
18,0227,0306,0227_9,0306_5,0.282521,0.270811,5078.155762,4444.170410,14.265550
19,0227,0306,0227_9,0306_10,0.406093,0.281338,5078.155762,3421.178955,48.432919
17,0227,0306,0227_9,0306_0,0.322274,0.279022,5078.155762,2632.532471,92.900024
10,0227,0306,0227_5,0306_2,0.338447,0.299433,4878.859863,5449.581055,11.697839
11,0227,0306,0227_6,0306_5,0.309760,0.271243,4873.996582,4444.170410,9.671685
12,0227,0306,0227_6,0306_10,0.348805,0.297352,4873.996582,3421.178955,42.465408
16,0227,0306,0227_8,0306_2,0.290177,0.252726,4041.062500,5449.581055,34.855156
0,0227,0306,0227_0,0306_0,0.293580,0.281686,3476.407715,2632.532471,32.055645
5,0227,0306,0227_2,0306_5,0.332398,0.290164,2886.754639,4444.170410,53.950405
